In [1]:
import time
import tweepy
import operator
import pandas as pd
from collections import defaultdict
import requests
import re
from bs4 import BeautifulSoup

from tweepy import OAuthHandler
# from get_config import get_config



# env = get_config()







In [2]:
access_token = "988745421584351232-4KmOVUHFjndSXxVMrah4g8MYMPM2cWE"
access_token_secret = "jSV86rHox9pNREKNJxbIFN1JFsXItOLny95WxA7KIOxKI"
consumer_key = "COA6fkjdCVLOFtepkapXZ5EbP"
consumer_secret = "NAXf8Q8ZPFKb46NcrWbREWtkL97yHku590ztp1Ebr5POoMSExz"


In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth)


In [6]:
def t_original_tweets(n):
	result = []
	count = 0
	for x in range(n):
		tweets = api.user_timeline(id="@ThisIsFusion",count=200,page=x+1,include_rts=True)
		tweets = [t for t in tweets if not t.text[:2] == 'RT']
		result += tweets
		count += 1
		if (x+1)%10 == 0:
			print ('sleep for one minute')
			time.sleep(60)
		print (count, 'of ', n, 'pages done')
	return result

In [7]:

def t_feature(level1,level2):
	if t.entities[level1]:
		return t.entities[level1][0][level2]
	else:
		return ''

In [8]:


def t_rank(n):
	tweets.sort(key=operator.attrgetter('retweet_count'), reverse=True)
	return [(t.id, t.retweet_count) for t in tweets[:n]]


In [9]:


def w_story_vedio(url):
	if url:
		return re.findall('http://fusion.net/(\w*)/\d*',url)[0]
	else:
		return ''


In [10]:

def w_idd(url):
	if url:
		return re.findall('http://fusion.net/\w*/(\d*)',url)[0]
	else:
		return ''

In [11]:

def convert_fusion_url(url):
	if url:
		r = requests.get(url)
		if re.search('http://fusion.net/(story)?(video)?/', r.url):
			return 'http://fusion.net/'+w_story_vedio(r.url)+'/'+w_idd(r.url)
	else:
		return ''


In [12]:
def w_short_json(url):
	if url:
		page = requests.get(url).text
		soup = BeautifulSoup(page)
		url = soup.find_all('link',{'rel':'canonical'})[0].get('href')
		return requests.get(url+'json').text
	else:
		return ''

In [13]:
# this API lacks some really important features
def w_long_json(url):
	wp = 'https://public-api.wordpress.com/rest/v1.1/sites/73194874/posts/'
	if url:
		return requests.get(wp+w_id(url).json())


In [14]:
def w_feature(pattern):
	if u:
		if re.findall(pattern,json):
			return re.findall(pattern,w_short_json(u))[0]
	else:
		return ''


In [15]:
# 3,200 most recent tweets (16 page search)
tweets = t_original_tweets(16)


1 of  16 pages done
2 of  16 pages done
3 of  16 pages done
4 of  16 pages done
5 of  16 pages done
6 of  16 pages done
7 of  16 pages done
8 of  16 pages done
9 of  16 pages done
sleep for one minute
10 of  16 pages done
11 of  16 pages done
12 of  16 pages done
13 of  16 pages done
14 of  16 pages done
15 of  16 pages done
16 of  16 pages done


The top most retweets

      id          frequency
(727247289891221504, 5)


The top most retweets

      id          frequency
(727247289891221504, 5)
tweets truncated to 1
importing twitter content...


NameError: name 't' is not defined

In [23]:
rank = t_rank(10)
print ("The top most retweets\n")
print ("      id          frequency")
for i in rank:
    print (i)


    tweets = tweets[:10]
    print ('tweets truncated to', len(tweets))


    print ("importing twitter content...")

    t_id = [t.id for t in tweets]
    t_url = [t_feature('urls','url') for t in tweets]
    t_text = [t.text for t in tweets]
    t_date = [t.created_at for t in tweets]
    t_hashtags = [t_feature('hashtags','text') for t in tweets]
    t_mentions = [t_feature('user_mentions','screen_name') for t in tweets]
    t_retweets = [t.retweet_count for t in tweets]
    t_favorites = [t.favorite_count for t in tweets]



print ("converting links from twitter to website...")

w_id = []
w_genre = []
w_title = []
w_link = []
w_date = []
w_authors = []
w_show = []
w_tags_section = []
w_tags_topic = []
w_tags_story_type = []
w_tags_location = []
w_tags_person = []
w_tags_organization = []
w_tags_event = []

fusion_urls = [convert_fusion_url(u) for u in t_url]



print ('importing website content...')

count = 0
for u in fusion_urls:
	if u:
		json = w_short_json(u)
		w_id.append(w_idd(u))
		w_genre.append(w_story_vedio(u))
		w_link.append(u)
		w_title.append(w_feature('"title":"(.*)","published"'))
		w_date.append(w_feature('"published":(.*)","link"'))
		w_authors.append(w_feature('"authors":\[(.*)\],"tags"'))
		w_show.append(w_feature('"show":"(.*)","images"'))
		w_tags_section.append(w_feature('"section":\[([\w", ]+)\]'))
		w_tags_topic.append(w_feature('"topic":\[([\w", ]+)\]'))
		w_tags_story_type.append(w_feature('"story_type":\[([\w", ]+)\]'))
		w_tags_location.append(w_feature('"location":\[([\w", ]+)\]'))
		w_tags_person.append(w_feature('"person":\[([\w", ]+)\]'))
		w_tags_organization.append(w_feature('"organization":\[([\w", ]+)\]'))
		w_tags_event.append(w_feature('"event":\[([\w", ]+)\]'))
	else:
		w_id.append('')
		w_genre.append('')
		w_link.append('')
		w_title.append('')
		w_date.append('')
		w_authors.append('')
		w_show.append('')
		w_tags_section.append('')
		w_tags_topic.append('')
		w_tags_story_type.append('')
		w_tags_location.append('')
		w_tags_person.append('')
		w_tags_organization.append('')
		w_tags_event.append('')
	count += 1
	print (count, 'data points imported')



print ('export to local file...')

tdic = {'t_id': t_id, 't_text': t_text, 't_date': t_date, 't_url': t_url, 't_hashtags': t_hashtags,
		't_mentions': t_mentions, 't_retweets': t_retweets, 't_favorites': t_favorites, 
		'w_id': w_id, 'w_genre': w_genre, 'w_title': w_title, 'w_link': w_link, 'w_date': w_date,
		'w_authors': w_authors, 'w_show': w_show, 'w_tags_section': w_tags_section,
		'w_tags_topic': w_tags_topic, 'w_tags_story_type': w_tags_story_type,
		'w_tags_location': w_tags_location, 'w_tags_person': w_tags_person,
		'w_tags_organization': w_tags_organization, 'w_tags_event': w_tags_event}


df = pd.DataFrame(tdic)

df.to_csv('top10.csv', encoding='utf-8')

print ('project is done!')



The top most retweets

      id          frequency
(727247289891221504, 5)
tweets truncated to 1
importing twitter content...


NameError: name 't' is not defined